Thabks to Marsh for his amazing notebook which helped me a lot! (https://www.kaggle.com/vbookshelf/skin-lesion-analyzer-tensorflow-js-web-app)

In this notebook I tried to classify the skin leason images using densenet to make a better result. I also edited the attention module compatable with this piece of code. But it takes too much time to train while using the attention module in kaggle so I had to give that up. I kept the attention module code so that if any one needs it the can take it from here as I had to stuggle to make this part run. Happy kaggeling! 


In [ ]:


import pandas as pd
import numpy as np
#import keras
#from keras import backend as K

import tensorflow
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation,LeakyReLU,Flatten,Input,Dense, Dropout , GlobalAveragePooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint,Callback

import os




# Pre-trained model
from tensorflow.keras.applications.densenet import DenseNet201

from tensorflow.keras.applications import EfficientNetB3




from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import shutil
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:

train_path = '../input/basedir/base_dir/base_dir/train_dir/'
valid_path = '../input/basedir/base_dir/base_dir/val_dir/'


In [ ]:
os.listdir('../input')


------------------------

In [ ]:
CFG = dict(
        batch_size        =  16,     # 8; 16; 32; 64; bigger batch size => moemry allocation issue
        epochs            =  100,   # 5; 10; 20;
        verbose           =   1,    # 0; 1
        workers           =   4,    # 1; 2; 3

        optimizer         = 'adam', # 'SGD', 'RMSprop'

        RANDOM_STATE      =  123,   
    
        # Path to save a model
        path_model        = '../working/',

        # Images sizes
        img_size          = 224, 
        img_height        = 224, 
        img_width         = 224, 

        # Images augs
        ROTATION          = 180.0,
        ZOOM              =  10.0,
        ZOOM_RANGE        =  [0.9,1.1],
        HZOOM             =  10.0,
        WZOOM             =  10.0,
        HSHIFT            =  10.0,
        WSHIFT            =  10.0,
        SHEAR             =   5.0,
        HFLIP             = True,
        VFLIP             = True,

        # Postprocessing
        label_smooth_fac  =  0.00,  # 0.01; 0.05; 0.1; 0.2;    
)

### Create the directory structure

In these folders we will store the images that will later be fed to the Keras generators. 

### Create Train and Val Sets

In [ ]:
df_data = pd.read_csv('../input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv')

df_data.head()

### Create a stratified val set

In [ ]:
df = df_data.groupby('lesion_id').count()
df = df[df['image_id'] == 1]
df.reset_index(inplace=True)
df.head()

In [ ]:

def identify_duplicates(x):
    
    unique_list = list(df['lesion_id'])
    
    if x in unique_list:
        return 'no_duplicates'
    else:
        return 'has_duplicates'
    
df_data['duplicates'] = df_data['lesion_id']
df_data['duplicates'] = df_data['duplicates'].apply(identify_duplicates)
df_data.head()

In [ ]:
df = df_data[df_data['duplicates'] == 'no_duplicates']
df.shape

In [ ]:
df_data.head()

In [ ]:
y = df['dx']
_, df_val = train_test_split(df, test_size=0.17, random_state=101, stratify=y)
df_val.shape


### Create a train set that excludes images that are in the val set

In [ ]:

def identify_val_rows(x):
    val_list = list(df_val['image_id'])
    if str(x) in val_list:
        return 'val'
    else:
df_data['train_or_val'] = df_data['image_id']
df_data['train_or_val'] = df_data['train_or_val'].apply(identify_val_rows)
   

    
    
    
df_train = df_data[df_data['train_or_val'] == 'train']



print(len(df_train))
print(len(df_val))


### Transfer the Images into the Folders

In [ ]:
df_data.set_index('image_id', inplace=True)

In [ ]:
print(len(os.listdir('../input/basedir/base_dir/base_dir/train_dir/nv')))
print(len(os.listdir('../input/basedir/base_dir/base_dir/train_dir/mel')))
print(len(os.listdir('../input/basedir/base_dir/base_dir/train_dir/bkl')))
print(len(os.listdir('../input/basedir/base_dir/base_dir/train_dir/bcc')))
print(len(os.listdir('../input/basedir/base_dir/base_dir/train_dir/akiec')))
print(len(os.listdir('../input/basedir/base_dir/base_dir/train_dir/vasc')))
print(len(os.listdir('../input/basedir/base_dir/base_dir/train_dir/df')))

### Visualize  augmented images

In [ ]:
print(len(os.listdir('../input/basedir/base_dir/base_dir/val_dir/nv')))
print(len(os.listdir('../input/basedir/base_dir/base_dir/val_dir/mel')))
print(len(os.listdir('../input/basedir/base_dir/base_dir/val_dir/bkl')))
print(len(os.listdir('../input/basedir/base_dir/base_dir/val_dir/bcc')))
print(len(os.listdir('../input/basedir/base_dir/base_dir/val_dir/akiec')))
print(len(os.listdir('../input/basedir/base_dir/base_dir/val_dir/vasc')))
print(len(os.listdir('../input/basedir/base_dir/base_dir/val_dir/df')))

### Set Up the Generators

In [ ]:
num_train_samples = len(df_train)
num_val_samples = len(df_val)
train_batch_size = 10
val_batch_size = 10
image_size = 224

train_steps = np.ceil(num_train_samples / train_batch_size)

val_steps = np.ceil(num_val_samples / val_batch_size)
print(train_steps,val_steps)
#908.0 94.0


In [ ]:

#change to densenet preprocess input
datagen = ImageDataGenerator(
    preprocessing_function= \
    tensorflow.keras.applications.densenet.preprocess_input)




train_batches = datagen.flow_from_directory(train_path,
                                            target_size=(image_size,image_size),
                                            batch_size=train_batch_size)

valid_batches = datagen.flow_from_directory(valid_path,
                                            target_size=(image_size,image_size),
                                            batch_size=val_batch_size)


test_batches = datagen.flow_from_directory(valid_path,
                                            target_size=(image_size,image_size),
                                            batch_size=1,
                                            shuffle=False)

# **Dense net**

In [ ]:
backbone = DenseNet121(weights='../input/densenet-keras/DenseNet-BC-121-32-no-top.h5', 
                     include_top=False,
                     input_shape=(224,224,3))

In [ ]:
    weights_path = "../input/efficentnet-b0b5-tensorflow-24-notop/efficientnet-b3_tf24_imagenet_1000_notop.h5"

    model = EfficientNetB3(include_top=False, weights=weights_path,  input_shape=(224,224,3))
    
    
    x = GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = BatchNormalization()(x)

    top_dropout_rate = 0.4
    x = Dropout(top_dropout_rate)(x)
    x = Dense(32, activation="relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(top_dropout_rate)(x)
    outputs = Dense(7, activation="sigmoid", name="pred")(x)
    

    # Compile
    model = Model(model.inputs, outputs, name="EfficientNet")

In [ ]:
model = Sequential()
model.add(backbone)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(7, activation='softmax'))

In [ ]:
model.summary()

### Train the Model

In [ ]:
from tensorflow.keras.metrics import categorical_accuracy

model.compile(Adam(learning_rate=0.001,decay=1e-5), loss='categorical_crossentropy',
              metrics=[categorical_accuracy])



In [ ]:
# Get the labels that are associated with each index
print(valid_batches.class_indices)

In [ ]:

 class_weights={
    0: 1.0, # akiec
    1: 1.0, # bcc
    2: 1.0, # bkl  
     3: 1.0, # df
     4: 3.0, # mel # Try to make the model more sensitive to Melanoma.
     5: 1.0, # nv
     6: 1.0, # vasc
 }

In [ ]:

filepath = "model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, 
                             save_best_only=True, mode='max')

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=2, 
                                   verbose=1, mode='max', min_lr=0.00001)
                              
                              
callbacks_list = [checkpoint, reduce_lr]

history = model.fit(train_batches, steps_per_epoch=train_steps,
                    validation_data=valid_batches,
                    validation_steps=val_steps,
                    epochs=20, verbose=1,
                   callbacks=callbacks_list)


### Evaluate the model using the val set

In [ ]:
model.save('effnet3.h5')


In [ ]:
# get the metric names so we can use evaulate_generator
model.metrics_names

In [ ]:
# Here the the last epoch will be used.

val_loss, val_cat_acc= \
model.evaluate(test_batches, 
                        steps=len(df_val))

print('val_loss:', val_loss)
print('val_cat_acc:', val_cat_acc)


In [ ]:
# Here the best epoch will be used.

model.save('model_backup.h5')
val_loss, val_cat_acc = \
model.evaluate(test_batches, 
                        steps=len(df_val))

print('val_loss:', val_loss)
print('val_cat_acc:', val_cat_acc)

### Plot the Training Curves

In [ ]:
# display the loss and accuracy curves

import matplotlib.pyplot as plt

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.figure()

plt.plot(epochs, acc, 'bo', label='Training cat acc')
plt.plot(epochs, val_acc, 'b', label='Validation cat acc')
plt.title('Training and validation cat accuracy')
plt.legend()
plt.figure()




plt.show()

### Create a Confusion Matrix

In [ ]:
# Get the labels of the test images.

test_labels = test_batches.classes

In [ ]:
# Print the label associated with each class
test_batches.class_indices

In [ ]:
# make a prediction
predictions = model.predict(test_batches, steps=len(df_val), verbose=1)

In [ ]:
# Source: Scikit Learn website
# http://scikit-learn.org/stable/auto_examples/
# model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-
# selection-plot-confusion-matrix-py


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()




In [ ]:
test_labels.shape

In [ ]:
# argmax returns the index of the max value in a row
cm = confusion_matrix(test_labels, predictions.argmax(axis=1))

In [ ]:
test_batches.class_indices

In [ ]:
# Define the labels of the class indices. These need to match the 
# order shown above.
cm_plot_labels = ['akiec', 'bcc', 'bkl', 'df', 'mel','nv', 'vasc']

plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')

### Generate the Classification Report

In [ ]:
# Get the index of the class with the highest probability score
y_pred = np.argmax(predictions, axis=1)

# Get the labels of the test images.
y_true = test_batches.classes
from sklearn.metrics import recall_score

recall_score(y_true, y_pred, average='macro')  

In [ ]:
from sklearn.metrics import classification_report

# Generate a classification report
report = classification_report(y_true, y_pred, target_names=cm_plot_labels)

print(report)

In [ ]:
%cd /kaggle/working
from IPython.display import FileLink
FileLink(r'model_backup.h5')

**Recall** = Given a class, will the classifier be able to detect it?<br>
**Precision** = Given a class prediction from a classifier, how likely is it to be correct?<br>
**F1 Score** = The harmonic mean of the recall and precision. Essentially, it punishes extreme values.

In [ ]:
img_path = "../input/basedir/base_dir/train_dir/mel/ISIC_0024310.jpg"
img = plt.imread(img_path)
plt.imshow(img)
plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.densenet import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image


img_path ="../input/melanoma-image/Melanoma.png"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_batch = np.expand_dims(img_array, axis=0)
img_preprocessed = preprocess_input(img_batch)
p = model.predict(img_preprocessed)
print(p)

In [ ]:
img_path ="../input/testing/bkl.png"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_batch = np.expand_dims(img_array, axis=0)
img_preprocessed = preprocess_input(img_batch)
p = model.predict(img_preprocessed)
print(p)


In [ ]:

img_path ="../input/testing/nevus.png"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_batch = np.expand_dims(img_array, axis=0)
img_preprocessed = preprocess_input(img_batch)
p = model.predict(img_preprocessed)
print(p)

In [ ]:

img_path ="../input/basedir/base_dir/train_dir/mel/ISIC_0024310.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_batch = np.expand_dims(img_array, axis=0)
img_preprocessed = preprocess_input(img_batch)
p = model.predict(img_preprocessed)
print(p)

In [ ]:
def dict_map(prediction):
    lesion_type_dict = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
    return dict(zip(lesion_type_dict,prediction))
dict_map(p[0])

In [ ]:
model.save("mo")